## Import and Load File

In [ ]:
import pandas as pd
import csv
import os
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import chi2_contingency
from pathlib import Path

#CSV Path
crime_reports_path = 'C:/Users/jrams/Desktop/Crime_Report.csv'
path = os.path.realpath(crime_reports_path)

#read file into df
crime_report_df = pd.read_csv(crime_reports_path, low_memory=False)

def all_data_df(crime_reports_path):                          
    df = pd.read_csv(crime_reports_path, low_memory=False)    
    return df

all_data_df = all_data_df(crime_reports_path)
all_data_df.head()

In [ ]:
#NEW DATAFRAME WITH ONLY CRIMES AND DATES
date_crime = pd.concat([all_data_df['Occurred Date'], all_data_df['Highest Offense Description']], axis=1)
#print(date_crime)

#LIST OF DIFFERENT CRIMES
crimes = date_crime['Highest Offense Description'].unique()
#print(crimes)

#FILTER ONLY AUTO THEFT CRIME BY DATE - DESCENDING
filtered_Auto_Theft_Crime = date_crime[date_crime['Highest Offense Description'].str.contains('Auto Theft', case=False)]
filtered_Auto_Theft_Crime_sorted = filtered_Auto_Theft_Crime.sort_values(by='Occurred Date', ascending=False)
print(filtered_Auto_Theft_Crime_sorted)


In [ ]:
#CONCATENATE DATA FRAMES

all_theft_crimes = pd.concat([filtered_Auto_Theft_Crime], ignore_index=True)

all_theft_crimes['Occurred Date'] = pd.to_datetime(all_theft_crimes['Occurred Date'], format='mixed')

all_theft_crimes['Season'] = pd.cut(                            #|Make bins
    all_theft_crimes['Occurred Date'].dt.dayofyear,             #|Take day of year from 'Occurred Date Column'
    bins= [0, 79, 171, 264, 355, 365],                          #|Create bins for new seasons column              
    labels=['Winter', 'Spring', 'Summer', 'Fall', 'Winter'],    #|Label bins   
    ordered=False                                               
    )

season_theft_crimes = all_theft_crimes.drop(columns=['Occurred Date'])

## Categorize Auto Theft by Season

In [ ]:
# TOTAL AUTO THEFT IN 2003-2023 PER SEASON

# Filter data 2003 to 2023
filtered_years = all_theft_crimes[(all_theft_crimes['Occurred Date'].dt.year >= 2003) & (all_theft_crimes['Occurred Date'].dt.year <= 2023)]

# Group by season and occurrences
season_counts = filtered_years.groupby('Season', observed=False).size()

# Do bar plot
season_counts.plot(kind='bar')

# Add labels and title
plt.title('Total Number of Auto Theft from 2003 to 2023 for Each Season')
plt.xlabel('Season')
plt.ylabel('Number of Auto Theft')

# Print plot
plt.show()

###############################################################################################################################################

# ADD SEASONAL AVERAGE

# Group by season and calculate the number of auto thefts per season
season_counts = filtered_years.groupby('Season', observed=False)['Occurred Date'].size()

# Calculate the average number of auto thefts per season
season_avg = season_counts.mean()

# Plot the data as a horizontal bar plot
plt.barh(season_counts.index, season_counts)

# Add a line for the average number of auto thefts
plt.axvline(x=season_avg, color='red', linestyle='--', label=f'Average: {season_avg:.2f}')

# Add labels and title
plt.title('Number of Auto Theft from 2003 to 2023 for Each Season')
plt.xlabel('Number of Auto Theft')
plt.ylabel('Season')
plt.legend()

# Show the plot
plt.show()

## Seasonal Trend & Hypothesis

In [ ]:
# Create a pivot table with years as rows and seasons as columns
seasonal_trends = filtered_years.pivot_table(index=filtered_years['Occurred Date'].dt.year, columns='Season', values='Highest Offense Description', aggfunc='count')

# Fill NaN values with 0
seasonal_trends.fillna(0, inplace=True)

# Plot the trends
plt.figure(figsize=(14, 7))
for column in seasonal_trends.columns:
    plt.plot(seasonal_trends.index, seasonal_trends[column], label=column)

plt.title('Seasonal Trends of Auto Theft Incidents Over the Years')
plt.xlabel('Year')
plt.ylabel('Number of Incidents')
plt.legend(title='Season')
plt.grid(True)
plt.show()

# Run hypothesis test
total = season_counts.sum()
expected = np.full(shape=season_counts.shape, fill_value=total / 4)  # Equal distribution among seasons
# Conducting the Chi-square test
chi2, p_value, dof, expected = chi2_contingency([season_counts.values, expected])
print(f"Chi-Square Statistic: {chi2}")
print(f"p-value: {p_value}")

#Reject Null Hypothesis - Seasons do impact auto theft

## Comparison of Occurences by Decade

In [ ]:
# Filter data for the years 2003 to 2013 and 2014 to 2023
filtered_years_2003_2013 = filtered_years[(filtered_years['Occurred Date'].dt.year >= 2003) & (filtered_years['Occurred Date'].dt.year <= 2013)].copy()
filtered_years_2014_2023 = filtered_years[(filtered_years['Occurred Date'].dt.year >= 2014) & (filtered_years['Occurred Date'].dt.year <= 2023)].copy()

# Extract decade from the date for each time range
filtered_years_2003_2013.loc[:, 'Decade'] = (filtered_years_2003_2013['Occurred Date'].dt.year // 100) + 1
filtered_years_2014_2023.loc[:, 'Decade'] = (filtered_years_2014_2023['Occurred Date'].dt.year // 100) + 1

# Group by decade and calculate the count of occurrences for each time range
decade_counts_2003_2013 = filtered_years_2003_2013.groupby('Decade').size()
decade_counts_2014_2023 = filtered_years_2014_2023.groupby('Decade').size()

# Plot the data as a bar plot
plt.bar(decade_counts_2003_2013.index - 0.2, decade_counts_2003_2013, width=0.4, label='2003-2013')
plt.bar(decade_counts_2014_2023.index + 0.2, decade_counts_2014_2023, width=0.4, label='2014-2023')

# Add labels and title
plt.title('Number of Auto Theft for Each Decade (2003-2013 vs 2014-2023)')
plt.xlabel('Decade')
plt.ylabel('Number of Auto Theft')
plt.xticks(decade_counts_2003_2013.index)

# Add legend
plt.legend()

# Show the plot
plt.show()

In [ ]:
#CALCULATE TOTAL NUMBER OF OCCURENCES IN EACH DECADE

# Filter data for the years 2003 to 2013
filtered_years_2003_2013 = filtered_years[(filtered_years['Occurred Date'].dt.year >= 2003) & (filtered_years['Occurred Date'].dt.year <= 2013)].copy()

# Calculate the total number of auto thefts for the period 2003-2013
total_2003_2013 = filtered_years_2003_2013.shape[0]

# Print the total number of occurrences
print(f"Total number of auto thefts from 2003 to 2013: {total_2003_2013}")

# Filter data for the years 2003 to 2013
filtered_years_2003_2013 = filtered_years[(filtered_years['Occurred Date'].dt.year >= 2014) & (filtered_years['Occurred Date'].dt.year <= 2023)].copy()

# Calculate the total number of auto thefts for the period 2003-2013
total_2014_to_2023 = filtered_years_2014_2023.shape[0]

# Print the total number of occurrences
print(f"Total number of auto thefts from 2014 to 2023: {total_2014_to_2023}")

In [ ]:
#CALCULATE INCREASE IN REPORTED OCCURENCES IN 2014-2023 COMPARED TO 2003-2013

# Given totals
total_2003_2013 = 25655
total_2014_2023 = 33733

# Calculate the percentage increase
percentage_increase = ((total_2014_2023 - total_2003_2013) / total_2003_2013) * 100

# Print the percentage increase
print(f"Percentage Increase in Auto Theft from 2003-2013 to 2014-2023: {percentage_increase:.2f}%")

# Data for the bar graph
labels = ['2003-2013', '2014-2023']
counts = [total_2003_2013, total_2014_2023]

# Create the bar plot
plt.figure(figsize=(8, 6))
bars = plt.bar(labels, counts, color='green')

# Annotate the bar plot with the total counts and percentage increase
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval + 500, f'{int(yval)}', ha='center', va='bottom')
    if bar.get_x() == bars[1].get_x():  # Check if it's the 2014-2023 bar
        plt.text(bar.get_x() + bar.get_width()/2, yval/2, f'{percentage_increase:.2f}%', ha='center', va='center', color='white', fontweight='bold')

# Add labels and title
plt.title('Total Auto Thefts (2003-2013 vs 2014-2023)')
plt.xlabel('Decade Period')
plt.ylabel('Number of Auto Thefts')

# Show the plot
plt.show()

In [ ]:
#DISPLAY # OF AUTO THEFTS IN 2003-2023

# Filter data for the years 2003 to 2013
filtered_years_2003_2013 = filtered_years[(filtered_years['Occurred Date'].dt.year >= 2003) & (filtered_years['Occurred Date'].dt.year <= 2013)].copy()

# Group by year and calculate the count of occurrences for each year
yearly_counts_2003_2013 = filtered_years_2003_2013.groupby(filtered_years_2003_2013['Occurred Date'].dt.year).size()

# Calculate the total number of auto thefts in the start year (2003) and end year (2013)
total_2003 = yearly_counts_2003_2013.loc[2003]
total_2013 = yearly_counts_2003_2013.loc[2013]

# Calculate the percentage increase
percentage_increase_2003_2013 = ((total_2013 - total_2003) / total_2003) * 100

# Print the percentage increase
print(f"Percentage of Auto Theft Occurences from 2003 to 2013: {percentage_increase_2003_2013:.2f}%")

# Plot the data as a line plot
plt.plot(yearly_counts_2003_2013.index, yearly_counts_2003_2013.values, marker='o', linestyle='-')

# Annotate the start and end year with the counts
plt.text(2003, total_2003, f"{total_2003}", ha='center', va='bottom')
plt.text(2013, total_2013, f"{total_2013}", ha='center', va='bottom')

# Add labels and title
plt.title('Number of Auto Thefts from 2003 to 2013')
plt.xlabel('Year')
plt.ylabel('Number of Auto Thefts')

# Show the plot
plt.show()

#######################################################################################################################################################

#DISPLAY # OF AUTO THEFTS IN 2014 - 2023

# Filter data for the years 2014 to 2023
filtered_years_2014_2023 = filtered_years[(filtered_years['Occurred Date'].dt.year >= 2014) & (filtered_years['Occurred Date'].dt.year <= 2023)].copy()

# Group by year and calculate the count of occurrences for each year
yearly_counts = filtered_years_2014_2023.groupby(filtered_years_2014_2023['Occurred Date'].dt.year).size()

# Calculate the total number of auto thefts in the start year (2014) and end year (2023)
total_2014 = yearly_counts.loc[2014]
total_2023 = yearly_counts.loc[2023]

# Calculate the percentage increase
percentage_increase_2014_2023 = ((total_2023 - total_2014) / total_2014) * 100

# Print the percentage increase
print(f"Percentage Increase in Auto Theft Occurences from 2014 to 2023: {percentage_increase_2014_2023:.2f}%")

# Plot the data as a line plot
plt.plot(yearly_counts.index, yearly_counts.values, marker='o', linestyle='-')

# Annotate the start and end year with the counts
plt.text(2014, total_2014, f"{total_2014}", ha='center', va='bottom')
plt.text(2023, total_2023, f"{total_2023}", ha='center', va='bottom')

# Add labels and title
plt.title('Number of Auto Thefts from 2014 to 2023')
plt.xlabel('Year')
plt.ylabel('Number of Auto Thefts')

# Show the plot
plt.show()



## Yearly Trend

In [ ]:
#Yearly Trend line

# Count the number of incidents per year
yearly_counts = filtered_years['Occurred Date'].dt.year.value_counts().sort_index()

# Create a line plot
plt.figure(figsize=(12, 6))
yearly_counts.plot(kind='line', marker='o', linestyle='-', color='purple')
plt.title('Yearly Trend of Auto Theft Incidents')
plt.xlabel('Year')
plt.ylabel('Number of Incidents')
plt.grid(True)
plt.show()